In [5]:
import os
import json
import traceback
import pandas as pd

In [9]:
from langchain_openai import ChatOpenAI

In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
KEY=os.getenv("OPENAI_API_KEY")

In [15]:
llm=ChatOpenAI(openai_api_key=KEY, model_name='gpt-4o-mini', temperature=0.5)

In [16]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x12212e090>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12212f9e0>, root_client=<openai.OpenAI object at 0x12193dac0>, root_async_client=<openai.AsyncOpenAI object at 0x11611bbf0>, model_name='gpt-4o-mini', temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [18]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [20]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs.
### RESPONSE_JSON
{response_json}

"""

In [21]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [22]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [24]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [25]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the questions and give a complete analysis of the quiz. Only use at max 50 words for complexity\
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that is perfectly fits the students abilities.
Quiz_MCQs:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [26]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", 'quiz'], template=TEMPLATE2)

In [27]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [30]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain, review_chain], 
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True,
)

In [31]:
file_path=r"/Users/predrag/Coding/mcqgen/data.txt"

In [32]:
file_path

'/Users/predrag/Coding/mcqgen/data.txt'

In [33]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [35]:
print(TEXT)

Srbija, zvanično Republika Srbija, država je na raskrsnici puteva srednje i jugoistočne Evrope u južnom delu Panonske nizije i središtu Balkanskog poluostrva.[5][6] Većim delom zahvata Balkansko poluostrvo, a manjim Panonsku niziju.[7] Srbija se na severu graniči sa Mađarskom, na severoistoku sa Rumunijom, na istoku sa Bugarskom, na jugu sa Severnom Makedonijom, na jugozapadu sa Albanijom i Crnom Gorom, a na zapadu sa Bosnom i Hercegovinom (entitetom Republika Srpska) i Hrvatskom. Broji oko osam miliona stanovnika.[8] Glavni i najveći grad je Beograd, koji spada među najstarije i najveće gradove u jugoistočnoj Evropi.[6][9] Zvanični jezik je srpski, a zvanična valuta srpski dinar.

Nakon slovenskog doseljavanja na Balkan (tokom 6. i 7. veka), Srbi su u ranom srednjem veku osnovali nekoliko država. Srednjovekovna Srbija je 1217. postala kraljevina, a vrhunac je dostigla 1346. proglašenjem carstva. Nakon turske najezde, Srpska despotovina je opstala do 1459, kada je potpala pod vlast Osm

In [36]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [37]:
NUMBER=5
SUBJECT="Republika Srbija"
TONE="simple"

In [38]:
# https://python.langchain.com/v0.1/docs/modules/model_io/llms/token_usage_tracking/

# How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

/var/folders/2m/c8pbt6811fx3v6t456w7r3cr0000gn/T/ipykernel_18184/1135891503.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Srbija, zvanično Republika Srbija, država je na raskrsnici puteva srednje i jugoistočne Evrope u južnom delu Panonske nizije i središtu Balkanskog poluostrva.[5][6] Većim delom zahvata Balkansko poluostrvo, a manjim Panonsku niziju.[7] Srbija se na severu graniči sa Mađarskom, na severoistoku sa Rumunijom, na istoku sa Bugarskom, na jugu sa Severnom Makedonijom, na jugozapadu sa Albanijom i Crnom Gorom, a na zapadu sa Bosnom i Hercegovinom (entitetom Republika Srpska) i Hrvatskom. Broji oko osam miliona stanovnika.[8] Glavni i najveći grad je Beograd, koji spada među najstarije i najveće gradove u jugoistočnoj Evropi.[6][9] Zvanični jezik je srpski, a zvanična valuta srpski dinar.

Nakon slovenskog doseljavanja na Balkan (tokom 6. i 7. veka), Srbi su u ranom srednjem veku osnovali nekoliko država. Srednjovekovna Srbija je 1217. postala kraljevina, a vrhunac je dostigla 1346. prog

In [39]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2173
Prompt Tokens:1545
Completion Tokens:628
Total Cost:0.00060855


In [40]:
response

{'text': 'Srbija, zvanično Republika Srbija, država je na raskrsnici puteva srednje i jugoistočne Evrope u južnom delu Panonske nizije i središtu Balkanskog poluostrva.[5][6] Većim delom zahvata Balkansko poluostrvo, a manjim Panonsku niziju.[7] Srbija se na severu graniči sa Mađarskom, na severoistoku sa Rumunijom, na istoku sa Bugarskom, na jugu sa Severnom Makedonijom, na jugozapadu sa Albanijom i Crnom Gorom, a na zapadu sa Bosnom i Hercegovinom (entitetom Republika Srpska) i Hrvatskom. Broji oko osam miliona stanovnika.[8] Glavni i najveći grad je Beograd, koji spada među najstarije i najveće gradove u jugoistočnoj Evropi.[6][9] Zvanični jezik je srpski, a zvanična valuta srpski dinar.\n\nNakon slovenskog doseljavanja na Balkan (tokom 6. i 7. veka), Srbi su u ranom srednjem veku osnovali nekoliko država. Srednjovekovna Srbija je 1217. postala kraljevina, a vrhunac je dostigla 1346. proglašenjem carstva. Nakon turske najezde, Srpska despotovina je opstala do 1459, kada je potpala p

In [42]:
quiz=response.get("quiz")

In [44]:
quiz=json.loads(quiz)

In [47]:
quiz

{'1': {'mcq': 'Koji je glavni grad Srbije?',
  'options': {'a': 'Novi Sad', 'b': 'Beograd', 'c': 'Niš', 'd': 'Kragujevac'},
  'correct': 'b'},
 '2': {'mcq': 'Koji je zvanični jezik u Srbiji?',
  'options': {'a': 'hrvatski',
   'b': 'mađarski',
   'c': 'srpski',
   'd': 'engleski'},
  'correct': 'c'},
 '3': {'mcq': 'Koje dve autonomne pokrajine se nalaze u sastavu Srbije?',
  'options': {'a': 'Vojvodina i Kosovo',
   'b': 'Vojvodina i Raška',
   'c': 'Kosovo i Metohija',
   'd': 'Kosovo i Sandžak'},
  'correct': 'a'},
 '4': {'mcq': 'Kada je Srbija postala kraljevina?',
  'options': {'a': '1217', 'b': '1346', 'c': '1878', 'd': '1882'},
  'correct': 'a'},
 '5': {'mcq': 'Koja je valuta u Srbiji?',
  'options': {'a': 'evro', 'b': 'dinara', 'c': 'dolar', 'd': 'forinta'},
  'correct': 'b'}}

In [48]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [49]:
quiz_table_data

[{'MCQ': 'Koji je glavni grad Srbije?',
  'Choices': 'a: Novi Sad | b: Beograd | c: Niš | d: Kragujevac',
  'Correct': 'b'},
 {'MCQ': 'Koji je zvanični jezik u Srbiji?',
  'Choices': 'a: hrvatski | b: mađarski | c: srpski | d: engleski',
  'Correct': 'c'},
 {'MCQ': 'Koje dve autonomne pokrajine se nalaze u sastavu Srbije?',
  'Choices': 'a: Vojvodina i Kosovo | b: Vojvodina i Raška | c: Kosovo i Metohija | d: Kosovo i Sandžak',
  'Correct': 'a'},
 {'MCQ': 'Kada je Srbija postala kraljevina?',
  'Choices': 'a: 1217 | b: 1346 | c: 1878 | d: 1882',
  'Correct': 'a'},
 {'MCQ': 'Koja je valuta u Srbiji?',
  'Choices': 'a: evro | b: dinara | c: dolar | d: forinta',
  'Correct': 'b'}]

In [50]:
pd.DataFrame(quiz_table_data)

,MCQ,Choices,Correct
0,Koji je glavni grad Srbije?,a: Novi Sad | b: Beograd | c: Niš | d: Kragujevac,b
1,Koji je zvanični jezik u Srbiji?,a: hrvatski | b: mađarski | c: srpski | d: eng...,c
2,Koje dve autonomne pokrajine se nalaze u sasta...,a: Vojvodina i Kosovo | b: Vojvodina i Raška |...,a
3,Kada je Srbija postala kraljevina?,a: 1217 | b: 1346 | c: 1878 | d: 1882,a
4,Koja je valuta u Srbiji?,a: evro | b: dinara | c: dolar | d: forinta,b


In [51]:
quiz=pd.DataFrame(quiz_table_data)

In [52]:
quiz.to_csv("srbija.csv", index=False)